In [ ]:
import ipywidgets as widgets
from glob import glob
import shutil as xu
import os
import time
import datetime

In [ ]:
class ziprecover():
    def __init__(self, src, dst, log_file):
        self.src = src
        self.dst = dst
        self.log_file = log_file
        self.process_start = time.time()
        self.copy_drive()
        self.process_end = time.time()
        self.process_elapsed = self.process_end - self.process_start
        self.write_log()

    def copy_drive(self):
        try:
            xu.copytree(self.src, self.dst)
        except xu.Error as e:
            self.errors = e

    def write_log(self):
        with open(self.log_file, "w+") as f:
            f.write("Geosciences ZIP Recovery Copy Script\n")
            f.write(datetime.datetime.now().isoformat() + "\n")
            f.write(f"Elapsed time (in seconds): {self.process_elapsed:0.1}\n")
            f.write("\n\n")
            f.write("Files errors:\n")
            if hasattr(self, "errors"):
                for ef in self.errors.args[0]:
                    f.write(f"{ef[0]} -- {ef[-1]}\n")

In [ ]:
dp = r"C:\Users\user\Pictures\Camera Roll"

disk_dir = r"C:\folder\IOMega_ZIP"
disk_template = "ZipDisk_{Disk:03}"

existing_disks = glob(f"{disk_dir}{os.sep}ZipDisk_*")
disk_folders = [x.split(os.sep)[-1] for x in existing_disks]
if len(disk_folders) != 0:
    highest_disk = max([int(x.split("_")[-1]) for x in disk_folders])
else:
    highest_disk = 0
print(highest_disk)

disk = widgets.IntText(value=highest_disk+1, description="Disk")

disk_button = widgets.Button(description="New Disk")
disk_handle = widgets.Output()

disk_photo_button = widgets.Button(description="disk label")
disk_photo_handle = widgets.Output()

copy_contents_button = widgets.Button(description="Copy Drive Contents")
copy_contents_handle = widgets.Output()
            
def disk_action(_):
    with disk_handle:
        new_disk_path = f"{disk_dir}{os.sep}ZipDisk_{disk.value:03}"
        print(new_disk_path)
        if os.path.exists(new_disk_path) == False:
            os.mkdir(new_disk_path)
            #os.mkdir(new_disk_path + os.sep + "Content")

def disk_photo_action(_):
    photos = glob(f"{dp}{os.sep}WIN_*.jpg")
    existing_labels = glob(f"{disk_dir}{os.sep}ZipDisk_{disk.value:03}{os.sep}ZipDisk_*_Label_*.jpg")
    if len(existing_labels) == 0:
        offset = 1
    else:
        existing_label_filenames = [x.split(os.sep)[-1] for x in existing_labels]
        offset = max([int(x.split("_")[-1].split(".")[0]) for x in existing_label_filenames]) + 1
    with disk_photo_handle:
        for photo in photos:
            new_photo_name = f"ZipDisk_{disk.value:03}_Label_{offset:02}.jpg"
            new_photo_path = f"{disk_dir}{os.sep}ZipDisk_{disk.value:03}{os.sep}{new_photo_name}"
            xu.move(photo, new_photo_path)
            offset += 1

def copy_contents_action(_):
    external_drive = f"D:{os.sep}"
    new_disk_path = f"{disk_dir}{os.sep}ZipDisk_{disk.value:03}"
    new_disk_content = f"{new_disk_path}{os.sep}Content"
    new_disk_log = f"{new_disk_path}{os.sep}transfer_log.txt"
    ziprecover(external_drive, new_disk_content, new_disk_log)
    
disk_button.on_click(disk_action)
disk_photo_button.on_click(disk_photo_action)
copy_contents_button.on_click(copy_contents_action)

display_box = widgets.VBox([disk, 
                            disk_button, disk_handle, disk_photo_button, disk_photo_handle,
                           copy_contents_button, copy_contents_handle])
display_box